In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.063611,True,32,1,37.0
1,4.687028,True,30,3,33.0
2,7.561856,True,20,1,39.0
3,7.561856,True,31,1,37.0
4,7.312733,True,32,1,40.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9997.000000,10000.000000,10000.000000,9886.000000
mean,7.303938,27.226500,1.033500,38.700890
std,1.313289,6.194646,0.192825,2.538734
min,0.595248,13.000000,1.000000,18.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.126239,32.000000,1.000000,40.000000
max,12.625874,52.000000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5096
False    4904
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)


In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [10]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6671 samples, validate on 742 samples
Epoch 1/10
6671/6671 [==============================] - 2s 228us/sample - loss: 5.0101 - mae: 1.3045 - mse: 5.0101 - val_loss: 1.4319 - val_mae: 0.9329 - val_mse: 1.4319
Epoch 2/10
6671/6671 [==============================] - 1s 133us/sample - loss: 1.4712 - mae: 0.9497 - mse: 1.4712 - val_loss: 1.6627 - val_mae: 1.0160 - val_mse: 1.6627
Epoch 3/10
6671/6671 [==============================] - 1s 108us/sample - loss: 1.4064 - mae: 0.9288 - mse: 1.4064 - val_loss: 1.5784 - val_mae: 0.9866 - val_mse: 1.5784
Epoch 4/10
6671/6671 [==============================] - 1s 119us/sample - loss: 1.3967 - mae: 0.9211 - mse: 1.3967 - val_loss: 1.3445 - val_mae: 0.9051 - val_mse: 1.3445
Epoch 5/10
6671/6671 [==============================] - 1s 116us/sample - loss: 1.3665 - mae: 0.9113 - mse: 1.3665 - val_loss: 1.3703 - val_mae: 0.9156 - val_mse: 1.3703
Epoch 6/10
6671/6671 [==============================] - 1s 112us/sample - loss: 1.3396 - mae: 0.9026 - 

In [11]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [12]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.5682507
Actual val:  8.3114272774

Predicted val:  6.70249
Actual val:  7.936641432

Predicted val:  5.7514997
Actual val:  6.2501051276999995

Predicted val:  6.3790154
Actual val:  8.313631900019999

Predicted val:  6.8865647
Actual val:  6.4374980503999994

Predicted val:  7.0335827
Actual val:  6.88062719702

Predicted val:  6.3601785
Actual val:  7.1870697412

Predicted val:  7.534616
Actual val:  8.1130112416

Predicted val:  6.6122284
Actual val:  7.43839671988

Predicted val:  6.140792
Actual val:  6.75055446244



In [13]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [14]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [15]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…